# Agnostic Simulator: the basics

This notebook talks about the abstract format used in this python package to represent a quantum circuit, and how we can then translate it to other formats or objects used in popular packages such as Qiskit, ProjectQ, Qulacs (...) and leverage the different features and performance they offer.

This functionality is pretty useful because it means that you can derive a given quantum circuit for any supported compute backend available with minimal effort, whether it is a classical simulator or an actual QPU. You therefore do not need to rewrite your program to run on a different platform, or to share your code in a specific format that your collaborators or clients expect, for publication or running an actual hardware experiment. It also means that whatever new method, program or library you develop can now suddenly run on all the compute backends available, enabling researchers and product users to benefit from your contributions regardless of the compute platform they intend to use.

## Table of contents
* [1. Abstract gate class](#1)
* [2. Abstract circuit class](#2)
* [3. Translator module](#3)
* [4. Simulator class](#4)
* [5. Beyond that](#end)

The scope of this package goes beyond this, as it will include techniques to optimize a quantum circuit or perform error-mitigation, for example. This notebook however only focuses on the basics: the abstract gate and circuit classes, as well as the `translator` and `simulator` modules. This should get you started with simulating a variety of quantum circuits, as well as computing expectation values of operators.


---


## Requirements

In order to run the contents of this notebook, I simply recommend that you install the agnostic simulator package as per the instructions, relying on the `setup.py` file in the github repository.

---

## 1. Abstract gate class <a class="anchor" id="1"></a>

This package is not here to provide yet another language to express a quantum circuit or to sell you an elaborate syntax to be used as a standard. Instead, it aims at providing users a straightforward and transparent way to represent a quantum gate operation, and by extension a quantum circuit.

The idea is to decouple quantum circuit definition, optimization, as well as post-processing, from the actual compute backend and state preparation: users are no longer necessarily tied to a given platform for their research or experiments and can easily jump from one to another depending on their needs. Do you need a faster simulator, or to use specific noise models only supported by specific backends? No need to rewrite your whole code.

The humble cornerstone of this package is the ability to represent a quantum gate operation acting on qubits as a simple python object. An abstract gate has:

- A name
- target qubit(s)  (only one qubit supported at the moment)
- control qubit(s) (only one supported at the moment)
- parameter(s)     (only one supported at the moment)
- a tag saying whether or not it is "variational", to keep track of gates with variational parameters


An abstract gate is pretty much just a dictionary, with the fields above. Some gates do not need all these fields to be populated in order to be clearly defined: not all of them require control qubits or parameters for instance. 

Let's have a look at how someone can define a gate and print it. You can see below that at instantiation, you can skip some fields or omit their name if the call is non-ambiguous. We always start with the name and the index of the target qubit, as all gates have them.

**Note**: qubit indexing starts at 0.


In [ ]:
from agnostic_simulator import Gate

# Create a Hadamard gate acting on qubit 2
H_gate = Gate("H", 2)
# Create a CNOT gate with control qubit 0 and target qubit 1
CNOT_gate = Gate("CNOT", target=1, control=0)
# Create a parameterized rotation on qubit 1 with angle 2 radians
RX_gate = Gate("RX", 1, parameter=2.)
# Create a parameterized rotation on qubit 1 , with an undefined angle, that will be tagged as variational
RZ_gate = Gate("RZ", 1, parameter="an expression", is_variational=True)
# Create a potato gate, acting on qubit 3.
POTATO_gate = Gate("POTATO", 3)

for gate in [H_gate, CNOT_gate, RX_gate, RZ_gate, POTATO_gate]:
    print(gate)

What's that, you work here and you don't even know about the POTATO gate?! Oof. Your imposter syndrom must be in full swing right now. It's ok. No one has noticed yet. Smile. Keep reading.

This abstract gate data-structure simply stores information, and does not need to fully specify a valid gate operation that correspond to, let's say, a very well-defined matrix representation of the operator. As you can see, the parameter for our `Rz` gate can be anything, and the actual existence of the POTATO gate is questionable. The gate set supported, the conventions on phases and parameters, are all backend-dependent. Therefore, the fields of your abstract gate only really need to make sense later on, once you have picked a target backend (Qiskit, qulacs...).

**Note**: The controlled gates, such as CNOT, expect that you first pass the target qubits and then the control ones, which may be counter-intuitive and trip you up.

## 2. Abstract circuit class <a class="anchor" id="2"></a>

An abstract circuit can be simply seen as a list of abstract gates. This class has a few methods to help users know at a glance how many gates or qubits are in a quantum circuit, or if it contains gates tagged as variational for example. In the future, other methods to compute the depth of the circuit, unentangled registers or qubits that do not actually need to be simulated could be added, for example.

Users can instantiate abstract circuits by directly passing a list of gates, or using the `add_gate` method on an existing abstract circuit object. It is also possible to concatenate abstract circuits using the $+$ operator: useful to build more complex circuits using simpler ones as building-blocks.

The number of qubits required by your circuit is automatically computed, based on the highest qubit index appearing in your gates. It is however possible to enforce a fixed-sized for your quantum circuit, that goes beyond this number.

We're all grown-ups, and technically you could perform surgery on your quantum circuit, to directly modify the fields of some of its gates. This can be useful for instance, to change the parameters used in "variational gates" (i.e gates tagged as variational) in the context of variational algorithms. Use responsibly.

In [ ]:
from agnostic_simulator import Circuit

# Here's a list of abstract gates
mygates = [Gate("H", 2), Gate("CNOT", 1, control=0), Gate("CNOT", target=2, control=1),
           Gate("Y", 0), Gate("RX", 1, parameter=2.)]

# Users can create empty circuit objects and use add_gate later on
circuit1 = Circuit()
for gate in mygates:
    circuit1.add_gate(gate)
    
# Users can also directly instantiate a circuit with a list of gates
circuit2 = Circuit(mygates)

# It is possible to concatenate abstract circuit objects
circuit3 = Circuit(mygates) + Circuit([Gate("RZ", 4, parameter="some angle", is_variational=True)])

# Printing a circuit prints gate list
print(circuit3)

# It is possible to examine properties of an abstract circuit directly
print(f"The number of gates contained in circuit3 is {circuit3.size}")
print(f"The number of qubits in circuit3 is {circuit3.width}")
print(f"Does circuit have gates tagged as variational? {circuit3.is_variational}")

# Even to have an overview of the type of gates it contains, and how many of them
print(f"Gate counts: {circuit3.counts}")

# Dark magic: update the parameters of the first variational gate in the circuit
circuit3._variational_gates[0].parameter = 777.
print(f"\n{circuit3}")

The abstract circuit can therefore be thought as a list of abstract gates, with some self-awareness: it knows what gate objects it contains, yet does not know what operation they actually implement on a quantum state. The latter is deferred to the translation step, which is different for each backend.

In the future, we may provide other features to optimize or analyze abstract circuits. The advantage of doing so means that whatever benefit come out of it, it will carry to any of the target backends.

**Note:** It is implied that qubits are measured in the computational basis at the end of the circuit. Users should NOT explicitly perform final measurements using the `MEASURE` instruction, which is designed to handle mid-circuit measurements, indicating that we intend to simulate a mixed state. As mixed states cannot be represented by statevectors, they are simulated differently and require to draw shots / samples, which may increase simulation time considerably.

### Helper functions to build circuits

This package provides helper functions allowing users to put together more general circuits easily, without the need to reinvent the wheel when it comes to common patterns. In the future, users could contribute functions generating useful for benchmarking reasons, building-blocks to form larger circuits, or well-known quantum circuits such as the QFT, for instance. These could be contributed as helper functions, or available in a separate folder aiming to gather a collection a circuits for different purposes. If you need to do something that seems fairly common, take a quick look around: you may have all the pieces already (or have the opportunity to design and contribute the pieces you need !).

**Note:** Most functions do not yield a `Circuit` object, but a list of gates (see the convention used at the end of the function name). For performance reasons, it is better to concatenate a list of gates and then turn it into a Circuit, than to turn smaller lists of gates into `Circuit` objects and then use the `+` operator to concatenate them. This will not be an issue unless you are for example running an algorithm that requires a circuit to be rebuilt frequently and requiring many `Circuit` objects to be concatenated.

Below, an example of how we can design a function to easily produce `Circuit` objects implementing a parameter sweep, working for any pauli measurement basis, which has been one of our most common hardware experiment so far in order to understand the capability of a quantum device. No need to write many different circuits explicitly when you can generate many variations of the same base circuit with a function and some simple loops over the range of interesting parameters !

The measurement basis is assumed to be passed as a list of 2-tuples of the form `(i, K)` where `i` is an integer denoting the qubit index and `K` a letter (string) that can take the value 'I', 'X', 'Y', or 'Z'; which also happens to be the format encountered while traversing a `QubitOperator` object in `Openfermion`.

In [ ]:
from agnostic_simulator.helpers.circuits import measurement_basis_gates, pauli_string_to_of

def theta_sweep(theta, m_basis):
    """ A single-parameter circuit, with change of basis at the end if needed """
    my_gates = [Gate('CNOT', target=0, control=1), 
                Gate('RX', target=1, parameter=theta), 
                Gate('CNOT', target=0, control=1)]
    my_gates += measurement_basis_gates(pauli_string_to_of(m_basis))
    return Circuit(my_gates)

# It is easy with agnostic simulator to move between string or Openfermion-style representations for Pauli words
for theta, m_basis in [(0.1, 'ZZ'), (0.2, 'ZZ'), (0.3, 'XY')]:
    c = theta_sweep(theta, m_basis)
    print(f"{c}\n")

## 3. Translator module <a class="anchor" id="3"></a>

In order to make use of the various compute backends available, the `translator` module provides users with functions that can translate from, and sometimes to, the abstract circuit format. This is the corner stone of this package: once the circuit has been translated, the user is free to use any of the methods and functionalities of the corresponding backend to analyze or simulate their circuit.

The `translator` module defines how the content of your abstract gates should be parsed, what gates are supported, the convention used, and is subject to error-checking: the contents must make sense for the target backend, otherwise you will get errors ! At that point, your gates must be supported and their parameters correct.

You can find in the translator module the dictionaries telling you what backends are supported, and what gates they currently support. As you can see, the exceptional `POTATO` gate is not yet supported by any available backend. You'd most certainly get an error if you tried to translate an abstract circuit containing such a gate, unless a major breakthrough in toaster oven technology happens and we integrate this disruptive compute platform.

In [ ]:
from agnostic_simulator.translator import SUPPORTED_GATES

for backend, gates in SUPPORTED_GATES.items():
    print(f'{backend} : {gates}')

The result of the translation step is specific to the given backend: some will return a quantum circuit object, some will return a string of instructions in their specific syntax & language, some a serialized JSON object...

**Note**: Do not use the `MEASURE` Gate in your noiseless/shotless simulations, unless your circuit requires a mid-circuit measurement. This gate is intended for the simulation of **mixed states**. 

**Note**: If you actually had a closer look at some simulation backends, such as Qiskit, Qulacs or ProjectQ, you'd find out that none of them have the same definition for the $R_z(\theta)$ operation: they all differ up to a phase or sign convention for $\theta$! Since the outcome of your simulation should not depend on the target backend, the translation step enforce a given convention, detailed in the documentation. In particular, this implies that the native circuit written for a given backend may not be equivalent to the same one written in abstract format, and then translated to this backend. Be mindful of that as you try to take some code written for a given backend, to port it to `agnostic_simulator`.

Below, we show how the translation function returns backend-specific objects, all with their usual built-in functionalities! See how the print function behaves differently for each of them for instance, and remember that you can use their built-in methods to accomplish many things (ex: after translating your circuit into a `Qiskit.QuantumCircuit` object, you can use the `draw` method to export it in a nice format: https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.draw.html)

In [ ]:
from agnostic_simulator.translator import translate_qulacs, translate_qiskit, translate_projectq, translate_qsharp, translate_openqasm

circ3_qulacs = translate_qulacs(circuit3)
print(f"{circ3_qulacs}\n")

circ3_qiskit = translate_qiskit(circuit3)
print(f"{circ3_qiskit}\n")

circ3_projectq = translate_projectq(circuit3)
print(f"{circ3_projectq}\n")

circ3_qdk = translate_qsharp(circuit3)
print(f"{circ3_qdk}\n")

circ3_openqasm = translate_openqasm(circuit3)
print(f"{circ3_openqasm}\n")

**Note**: there exists a "limited" translation function from OpenQASM back to abstract format, whose purpose is detailed below. 

### Saving, loading and sharing quantum circuits

The abstract circuit class used by the agnostic simulator package can be translated into various formats and objects, so that users can easily save, load and share them with collaborators without providing an explicit code to do so. This is particularly relevant if you are working with external collaborators that do not expect to receive a circuit in an unknown format, internal to 1QBit, or if the circuit you want to share comes at the expense of complex calculations (using an application library, or maybe as a result of a variational or iterative procedure).

A first option is to use the function translating abstract format to OpenQASM, which allows you to export your circuit in a straightforward OpenQASM 2.0 text format (it relies on the QASM export from the IBM Qiskit package, as they drive the standard), which is both human readable and can be imported by other quantum circuit simulation packages in general. A "reverse" translation from OpenQASM back to abstract format is available as well, but only supports a subset of OPENQASM 2.0. Users can thus save as QASM and load this back into an abstract circuit as well. No need to rewrite anything.

A second option is to first translate your quantum circuit into the target format (string, object) and write it to file. For non-string objects, users may consider using a Python package such as `pickle` to serialize and save/load from a `.pkl` file a quantum circuit object. Your collaborators should then be able to load the circuit into their code by reading the string from file, or reading the pickled objects using the Python `pickle` package. I recommand to give them a sample code to load the objects properly.

## 4.  Simulator class <a class="anchor" id="4"></a>

The `Simulator` class provides a common interface to all compute backends, allowing users to focus on the high-level details of their simulation (number of shots, noise model...) without writing low-level code for the target backend. Instead, the `Simulator` object handles all translation and quantum circuit simulation steps, directly returning the results to the user. Some post-processing methods, such as the computation of the expectation value of a qubit operator, are available as well.

Not all backends provide the same features: some of them do not give access to a state-vector representation of the quantum state, or do not support noisy simulation for instance. Just like the `Translator` module, the `Simulator` module provides a data-structure containing the currently supported features and characteristics of some backends.

In [ ]:
from agnostic_simulator import Simulator, backend_info

for backend, info in backend_info.items():
    print(f'{backend} : {info}')

### Noiseless simulation

In the example below, we show how the `Simulator` class allows for switching backends easily, to perform quantum circuit simulation. We first show how to perform noiseless simulation with access to the exact amplitudes through the state-vector representation of the quantum state, and then show how a noiseless shot-based simulation can be performed.

The `simulate` method returns a 2-tuple:

- the first entry is a sparse histogram of frequencies associated to the different observed states, in least-significant qubit first order (e.g '01' means qubit 0 (resp 1) measured in $|0>$ (resp $|1>$) state). That is, it is to be read "left-to-right" in order to map each qubit to the basis state it was observed in.
- the second entry contains the state-vector representation of the quantum state, if available on the target backend and if the user requires it using the `return_statevector` optional parameter. Watch out for the order of the amplitudes in the state vector: the `backend_info` data-structure described above helps with understanding how they map to the different basis states.

The `simulate` method can also take the optional parameter `initial_statevector`, which allows to start the system in a given state and "resume" a simulation. It can in particular be useful to avoid resimulating something again and again, when the result is already known and could just be loaded instead. This is one of the perks of statevector simulators !

In [ ]:
# Create a circuit object in abstract format
c = Circuit([Gate("RX", 0, parameter=2.), Gate("RY", 1, parameter=-1.)])

# ProjectQ noiseless simulator (no shot count or noise model specified)
sim_projectq = Simulator(target="projectq")
print(sim_projectq.simulate(c))

# Qiskit noiseless simulator (no shot count or noise model specified)
sim_qiskit = Simulator(target="qiskit")
print(sim_qiskit.simulate(c))

# Qulacs noiseless simulator (no shot count or noise model specified)
# Ask to return the state vector as well (exposes the complex amplitudes)
sim_qulacs = Simulator(target="qulacs")
print(f"\n{sim_qulacs.simulate(c, return_statevector=True)}")

**Note**: The native equivalents of this abstract circuit in Qulacs and Qiskit would not return the same results. This is due to the fact that $Rx_{qiskit}(\theta) = Rx_{qulacs}(-\theta)$ (e.g sign convention), which is also true for the $Ry$ and $Rz$ gates. This package however ensures consistent behavior across backends, by enforcing a specifying convention in the translator. For more information about conventions used, please refer to the documentation or check out the source code of the `translator` module.

In the above cell, we instantiated the `Simulator` class without specifying a number of shots or a noise model, but it is possible to do so. You can tweak the behavior of the `Simulator` object by modifying -some- of its attributes after it has been instantied. In particular:

- `freq_threshold` is the threshold used to discard negligible frequencies from the histogram returned by the `simulate` method, in order to avoid returning $2^{n\_qubits}$ numbers and focus on the main observables.
- `n_shots` and `noise_model` can be changed after the class has been instantiated

In the cell below, we can see how instantiating a shot-based simulator and increasing shot count yields results that are getting closer to the exact theoretical distribution.

In [ ]:
# Exact probabilities
sim_qulacs = Simulator(target="qulacs")
exact_freqs, _ = sim_qulacs.simulate(c)
print(exact_freqs)

# Approximation with different number of shots (higher=more accurate)
sim_qulacs_shots = Simulator(target="qulacs", n_shots=100)
freqs, _ = sim_qulacs_shots.simulate(c)
print(freqs)

sim_qulacs_shots.n_shots=10**4
freqs, _ = sim_qulacs_shots.simulate(c)
print(freqs)

sim_qulacs_shots.n_shots=10**6
freqs, _ = sim_qulacs_shots.simulate(c)
print(freqs)

### Mixed states

The abstract circuit format provides a `MEASURE` instruction, supported by most compute backends, with the intent of simulating mixed states / mid-circuit measurements in the computational basis (e.g along the Z axis). As mixed states cannot be represented by a statevector, the statevector simulator backends default to simulating the quantum circuit with shots, in order to return a histogram of frequencies. Users thus must ensure the `n_shots` attribute of their `Simulator` object has been set.

Simulating a mixed state can be considerably slower than simulating a pure state (some backends, such as ProjectQ, are particularly NOT good at this). Including final measurements in your state-preparation circuit is not recommended, if you intend to use `agnostic_simulator` to simulate it.

In [ ]:
# This circuit prepares a Bell pair (superposition of states |00> and |11>), then measures and flips qubit 0.
# Depending on the result of the measurement, the statevector describing the quantum state would be different.
circuit_mixed = Circuit([Gate("H", 0), Gate("CNOT", target=1, control=0), Gate("MEASURE", 0), Gate("X", 0)])
sim = Simulator(target="qulacs", n_shots=10**5)

freqs, _ = sim.simulate(circuit_mixed)
print(freqs)

In the future, this package may provide compute backends that are more appropriate to simulate mixed states, relying on density matrices for example.

### Expectation values

The `get_expectation_value`method can be used to compute the expectation value of an Openfermion-style operator with regards to a state-preparation circuit, and `get_expectation_value_from_frequencies_oneterm` can directly take a sparse histogram of frequencies obtained by simulating a quantum circuit on a backend (which could be resulting from a QPU experiment). 

**Note**: `get_expectation_value_from_frequencies_oneterm` is a static method that does not require a `Simulator` object to be instantiated: it can be called directly. For now, users are to manually loop over terms and corresponding histograms if they intend to compute the expectation value of a linear combination of operators. 

In [ ]:
# Openfermion operators can be used
from openfermion.ops import QubitOperator
op = 1.0 * QubitOperator('Z0')

# Option1: Directly through a simulator backend, providing the state-preparation circuit
sim = Simulator("qulacs")
expval1 = sim.get_expectation_value(op, c)
print(expval1)

# Option2: Assume quantum circuit simulation was performed separately (by this package or different services)
freqs, _ = sim.simulate(c)  # circuit c must be consistent with operator, regarding measurement along non-z axes
term, coef = tuple(op.terms.items())[0]  # This yields ((0, 'Z'),), 1.0
expval2 = coef * Simulator.get_expectation_value_from_frequencies_oneterm(term, freqs)
print(expval2)

## Beyond that <a class="anchor" id="end"></a>

This notebook provided a general introduction to the agnostic simulator package. I hope you liked it.
To dive into other topics, such as **noisy simulation** or how to use this package for **variational algorithms** for example, please refer to the other examples and tutorials available !

---

*Valentin Senicourt*